# 最初から機械学習でやってみる

In [1]:
import os
import json
import time
import glob
from typing import TypeVar, Type, List, Optional
from dataclasses import dataclass
from pydantic import BaseModel
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from openai import AzureOpenAI
from tqdm import tqdm
from typing import List, Dict, Optional, Any, Tuple

In [2]:
csv_files = glob.glob('*.csv')
print(csv_files)

['extracted_500_first_20241130_検索結果1204.csv', 'pubmed_true_data.csv', 'exported_data500.csv', 'remaining_data10404.csv', 'exported_data500_スクリーニング結果1210.csv']


In [3]:
df = pd.read_csv(csv_files[4])
# Assuming your dataframe is named 'df'
df['included'] = df['notes'].astype(str).str.contains('Included')

print(df['included'].value_counts())
print(f'不一致の数: {df[df["included"] == True]["notes"].str.contains("Excluded").sum()}')
#予測の元となる変数

import json

def create_json_text(row):
    return json.dumps({
        'title': str(row['title']),
        'abstract': str(row['abstract'])
    })

df['tiab'] = df.apply(create_json_text, axis=1)

included
False    473
True      27
Name: count, dtype: int64
不一致の数: 0


In [10]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix, classification_report
import optuna
from lightgbm import LGBMClassifier

def prepare_text_data(df, text_column='tiab', label_column='included'):
    """テキストデータの前処理とTF-IDF変換"""
    texts = df[text_column].fillna('')

    vectorizer = TfidfVectorizer(
        max_features=10000,
        min_df=2,
        max_df=0.95,
        ngram_range=(1, 2)
    )

    X_vec = vectorizer.fit_transform(texts)
    y = df[label_column]

    class_weights = dict(zip(
        y.unique(),
        [1 / (len(y) * (y == label).mean()) for label in y.unique()]
    ))

    return X_vec, y, vectorizer, class_weights

def fbeta_score_custom(y_true, y_pred, beta=1):
    """カスタムF-betaスコアの計算"""
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred, labels=[0,1]).ravel()
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0.0
    if precision == 0.0 and recall == 0.0:
        return 0.0
    fbeta = (1 + beta**2) * (precision * recall) / ((beta**2 * precision) + recall)
    return fbeta

def calculate_detailed_metrics(y_true, y_pred):
    """詳細な評価指標の計算と表示"""
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred, labels=[0,1]).ravel()

    sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    f1 = 2 * (precision * sensitivity) / (precision + sensitivity) if (precision + sensitivity) > 0 else 0

    print("\n=== 詳細な評価指標 ===")
    print(f"感度 (Sensitivity/Recall): {sensitivity:.3f}")
    print(f"特異度 (Specificity): {specificity:.3f}")
    print(f"適合率 (Precision): {precision:.3f}")
    print(f"F1スコア: {f1:.3f}")

    print("\n=== 混同行列 ===")
    print("                  Predicted")
    print("                  Negative  Positive")
    print(f"Actual Negative    {tn:^8} {fp:^8}")
    print(f"      Positive    {fn:^8} {tp:^8}")

    return {
        'sensitivity': sensitivity,
        'specificity': specificity,
        'precision': precision,
        'f1': f1,
        'confusion_matrix': {'tn': tn, 'fp': fp, 'fn': fn, 'tp': tp}
    }

def evaluate_model(X_vec, y, params, threshold, class_weights, beta=2):
    """モデルの評価（交差検証）"""
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    scores = []
    all_predictions = []
    all_true_values = []

    for train_idx, val_idx in skf.split(X_vec, y):
        X_tr = X_vec[train_idx]
        X_val = X_vec[val_idx]
        y_tr = y.iloc[train_idx]
        y_val = y.iloc[val_idx]

        model = LGBMClassifier(
            **params,
            random_state=42,
            class_weight=class_weights
        )

        model.fit(X_tr, y_tr)
        y_prob = model.predict_proba(X_val)[:, 1]
        y_pred = (y_prob >= threshold).astype(int)

        score = fbeta_score_custom(y_val, y_pred, beta=beta)
        scores.append(score)

        all_predictions.extend(y_pred)
        all_true_values.extend(y_val)

    if len(scores) == 5:
        print("\n=== 交差検証全体の評価 ===")
        metrics = calculate_detailed_metrics(all_true_values, all_predictions)
        print("\nClassification Report:")
        print(classification_report(all_true_values, all_predictions))

    return np.mean(scores)

def objective(trial, X_vec, y, class_weights, beta=2):
    """Optunaの目的関数"""
    params = {
        "num_leaves": trial.suggest_int("num_leaves", 8, 128),
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.3, log=True),
        "n_estimators": trial.suggest_int("n_estimators", 50, 500),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-3, 10.0, log=True),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
        "verbosity": -1
    }
    threshold = trial.suggest_float("threshold", 0.1, 0.9)

    score = evaluate_model(X_vec, y, params, threshold, class_weights, beta=beta)
    return score

def train_final_model(df, beta_value=2, n_trials=50):
    """完全なトレーニングパイプライン"""
    # データの準備
    X_vec, y, vectorizer, class_weights = prepare_text_data(df)

    print(f"データセットの形状: {X_vec.shape}")
    print(f"クラスの分布:\n{y.value_counts(normalize=True)}")
    print(f"クラスの重み: {class_weights}")

    # ハイパーパラメータの最適化
    study = optuna.create_study(direction="maximize")
    study.optimize(
        lambda trial: objective(trial, X_vec, y, class_weights, beta=beta_value),
        n_trials=n_trials
    )

    best_params = study.best_trial.params
    threshold = best_params.pop("threshold")

    print("\nBest parameters:", best_params)
    print("Best threshold:", threshold)

    final_score = evaluate_model(X_vec, y, best_params, threshold, class_weights, beta=beta_value)
    print(f"\nFinal Mean F{beta_value} Score:", final_score)

    # 最終モデルの学習
    final_model = LGBMClassifier(
        **best_params,
        random_state=42,
        class_weight=class_weights
    )
    final_model.fit(X_vec, y)

    # 最終評価
    print("\n=== 最終モデルの評価（全データ） ===")
    y_prob = final_model.predict_proba(X_vec)[:, 1]
    y_pred = (y_prob >= threshold).astype(int)
    final_metrics = calculate_detailed_metrics(y, y_pred)

    return final_model, vectorizer, threshold, best_params

def predict_new_text(text, model, vectorizer, threshold):
    """新しいテキストの予測"""
    X_new = vectorizer.transform([text])
    prob = model.predict_proba(X_new)[0, 1]
    prediction = prob >= threshold
    return prediction, prob



In [11]:
# 使用例
if __name__ == "__main__":
    # データの読み込み
    #df = pd.read_csv('your_data.csv')  # tiab列とincluded列を含むデータ

    # モデルのトレーニング
    model, vectorizer, threshold, best_params = train_final_model(
        df,
        beta_value=4,  # より高い値で再現率を重視
        n_trials=50
    )


[I 2024-12-11 07:20:16,500] A new study created in memory with name: no-name-05183813-9a77-4c2a-8aee-b5a9b0f2e8c1


データセットの形状: (500, 10000)
クラスの分布:
included
False    0.946
True     0.054
Name: proportion, dtype: float64
クラスの重み: {np.False_: np.float64(0.0021141649048625794), np.True_: np.float64(0.037037037037037035)}


[I 2024-12-11 07:20:18,952] Trial 0 finished with value: 0.6678733424377314 and parameters: {'num_leaves': 37, 'learning_rate': 0.07045554768608817, 'n_estimators': 402, 'reg_lambda': 0.026739358097090483, 'min_child_samples': 42, 'threshold': 0.13010222312353817}. Best is trial 0 with value: 0.6678733424377314.



=== 交差検証全体の評価 ===

=== 詳細な評価指標 ===
感度 (Sensitivity/Recall): 0.704
特異度 (Specificity): 0.928
適合率 (Precision): 0.358
F1スコア: 0.475

=== 混同行列 ===
                  Predicted
                  Negative  Positive
Actual Negative      439       34   
      Positive       8        19   

Classification Report:
              precision    recall  f1-score   support

       False       0.98      0.93      0.95       473
        True       0.36      0.70      0.47        27

    accuracy                           0.92       500
   macro avg       0.67      0.82      0.71       500
weighted avg       0.95      0.92      0.93       500



C:\Users\youki\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\youki\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\youki\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} i

...


=== 交差検証全体の評価 ===

=== 詳細な評価指標 ===
感度 (Sensitivity/Recall): 1.000
特異度 (Specificity): 0.000
適合率 (Precision): 0.054
F1スコア: 0.102

=== 混同行列 ===
                  Predicted
                  Negative  Positive
Actual Negative       0       473   
      Positive       0        27   

Classification Report:
              precision    recall  f1-score   support

       False       0.00      0.00      0.00       473
        True       0.05      1.00      0.10        27

    accuracy                           0.05       500
   macro avg       0.03      0.50      0.05       500
weighted avg       0.00      0.05      0.01       500



[I 2024-12-11 07:21:43,760] Trial 48 finished with value: 0.684548664612836 and parameters: {'num_leaves': 79, 'learning_rate': 0.03587619162050904, 'n_estimators': 125, 'reg_lambda': 0.02330293076838238, 'min_child_samples': 45, 'threshold': 0.3362345279738537}. Best is trial 31 with value: 0.7889312998813136.



=== 交差検証全体の評価 ===

=== 詳細な評価指標 ===
感度 (Sensitivity/Recall): 0.741
特異度 (Specificity): 0.903
適合率 (Precision): 0.303
F1スコア: 0.430

=== 混同行列 ===
                  Predicted
                  Negative  Positive
Actual Negative      427       46   
      Positive       7        20   

Classification Report:
              precision    recall  f1-score   support

       False       0.98      0.90      0.94       473
        True       0.30      0.74      0.43        27

    accuracy                           0.89       500
   macro avg       0.64      0.82      0.69       500
weighted avg       0.95      0.89      0.91       500



[I 2024-12-11 07:21:45,345] Trial 49 finished with value: 0.4811461412151067 and parameters: {'num_leaves': 57, 'learning_rate': 0.02633184156596739, 'n_estimators': 273, 'reg_lambda': 0.08432237047317603, 'min_child_samples': 57, 'threshold': 0.6335684729987513}. Best is trial 31 with value: 0.7889312998813136.



=== 交差検証全体の評価 ===

=== 詳細な評価指標 ===
感度 (Sensitivity/Recall): 0.481
特異度 (Specificity): 0.975
適合率 (Precision): 0.520
F1スコア: 0.500

=== 混同行列 ===
                  Predicted
                  Negative  Positive
Actual Negative      461       12   
      Positive       14       13   

Classification Report:
              precision    recall  f1-score   support

       False       0.97      0.97      0.97       473
        True       0.52      0.48      0.50        27

    accuracy                           0.95       500
   macro avg       0.75      0.73      0.74       500
weighted avg       0.95      0.95      0.95       500


Best parameters: {'num_leaves': 64, 'learning_rate': 0.025999478014992004, 'n_estimators': 248, 'reg_lambda': 0.0998410986638099, 'min_child_samples': 43}
Best threshold: 0.30117219313875077

=== 交差検証全体の評価 ===

=== 詳細な評価指標 ===
感度 (Sensitivity/Recall): 0.889
特異度 (Specificity): 0.869
適合率 (Precision): 0.279
F1スコア: 0.425

=== 混同行列 ===
                  Predicted
       

In [12]:

import joblib

# 日時をstringsで
timestamp = time.strftime('%Y%m%d%H%M%S')

# モデルと関連オブジェクトの保存
model_data = {
    'model': model,
    'vectorizer': vectorizer,
    'threshold': threshold,
    'best_params': best_params
}
joblib.dump(model_data, f'{timestamp}_v2_model_data.joblib')
print("\nモデルと関連オブジェクトを保存しました。")


モデルと関連オブジェクトを保存しました。


In [13]:
models = glob.glob('*.joblib')
print(models)

['model_data.joblib', '20241207084511_model_data.joblib', '20241211072208_v2_model_data.joblib']


In [14]:
# 保存したモデルデータをロード
model_data = joblib.load(models[2])

# 各オブジェクトの取り出し
loaded_model = model_data['model']
loaded_vectorizer = model_data['vectorizer']
loaded_threshold = model_data['threshold']
loaded_best_params = model_data['best_params']

print("モデルと関連オブジェクトをロードしました。")


モデルと関連オブジェクトをロードしました。


# 絶対に落としてはいけない文献を落とさないことを確認

In [15]:
csv_files

['extracted_500_first_20241130_検索結果1204.csv',
 'pubmed_true_data.csv',
 'exported_data500.csv',
 'remaining_data10404.csv',
 'exported_data500_スクリーニング結果1210.csv']

In [16]:
df2 = pd.read_csv(csv_files[1])
df2.columns = ["PMID", "title", "abstract"]
df2["tiab"] = df2.apply(create_json_text, axis=1)

In [17]:
df2.shape

(10, 4)

In [18]:
def predict_and_summarize(df, create_json_text, loaded_vectorizer, loaded_model, loaded_threshold):
    # "tiab"列の作成 (DataFrame dfに対して適用)
    df["tiab"] = df.apply(create_json_text, axis=1)

    # テキストデータの前処理とベクトル化
    X_new = loaded_vectorizer.transform(df["tiab"].fillna(''))

    print(f"新しいデータの形状: {X_new.shape}")

    # 予測確率の取得
    y_prob_new = loaded_model.predict_proba(X_new)[:, 1]

    # 閾値を適用してクラスラベルを決定
    y_pred_new = (y_prob_new >= loaded_threshold).astype(int)

    # 予測結果をデータフレームに追加
    df['prediction'] = y_pred_new
    df['probability'] = y_prob_new

    print("予測を実行し、結果をデータフレームに追加しました。")

    # 予測結果の表示
    print("\n=== 予測結果 ===")
    print(df[['tiab', 'prediction', 'probability']])

    # クラス分布の確認
    class_dist = df['prediction'].value_counts()
    print("\nクラスの分布:")
    print(class_dist)

    # 確率の統計情報
    prob_stats = df['probability'].describe()
    print("\n予測確率の統計情報:")
    print(prob_stats)

    # 必要に応じて、更新後のdfや統計情報を返す
    return df, class_dist, prob_stats


In [19]:
# 必要なオブジェクトを準備
# df2, create_json_text, loaded_vectorizer, loaded_model, loaded_threshold を用意済みとする
updated_df, class_distribution, probability_stats = predict_and_summarize(
    df2, create_json_text, loaded_vectorizer, loaded_model, loaded_threshold
)


新しいデータの形状: (10, 10000)
予測を実行し、結果をデータフレームに追加しました。

=== 予測結果 ===
                                                tiab  prediction  probability
0  {"title": "A multicenter, randomized, controll...           1     0.520263
1  {"title": "Transfusion requirements after head...           1     0.734563
2  {"title": "Restrictive vs Liberal Transfusion ...           1     0.318891
3  {"title": "Liberal or Restrictive Transfusion ...           1     0.341484
4  {"title": "Effect of Transfusion on Mortality ...           1     0.306149
5  {"title": "Transfusion-related risk of seconda...           1     0.770659
6  {"title": "Impact of Blood Product Transfusion...           1     0.529030
7  {"title": "Transfusion practice and blood stre...           1     0.427367
8  {"title": "Anemia, transfusions and hospital o...           1     0.862065
9  {"title": "Transfusion of platelets, but not o...           1     0.793162

クラスの分布:
prediction
1    10
Name: count, dtype: int64

予測確率の統計情報:
count    10.0

# 残っているファイルに対して予測を行い、Trueであったものだけを残す

'remaining_data10404.csv'

In [22]:
df3 = pd.read_csv(csv_files[3])

In [23]:
df3.head()

,key,title,authors,journal,volume,issue,pages,abstract,year,publisher,url,issn
0,03fffde5-8aca-49b6-8666-d98a2ba3d4ad,Association of transfusion red blood cell stor...,"[{'author': 'Yamal, José-Miguel', 'author_abbr...",The journal of trauma and acute care surgery,79,5,843-9,BACKGROUND: The effect of red blood cell (RBC)...,2015,United States,https://doi.org/10.1097/TA.0000000000000834,"{""pmid"": 26496111, ""electronic_issn"": ""2163-07..."
1,4944c10a-6c8f-4cd3-a1c8-838fff9273d1,Transfusion of packed red blood cells does not...,"['Mas, A.', 'Roig, I.', 'Baigorri, F.', 'Martí...",Medicina Intensiva,20,7,327-332,Background. The usefulness of stored red blood...,1996/09//,"Ediciones Doyma, S.L., Travesera de Gracia 17-...",['https://go.openathens.net/redirector/rsm.ac....,"{""accession_number"": ""685936757; 26332003"", ""t..."
2,0d828515-1129-4a05-ae50-14a360c1f16b,Red blood cell transfusion in the critically i...,"[{'author': 'Lelubre, Christophe', 'author_abb...",Annals of intensive care,1,NaN,43,Red blood cell (RBC) transfusion is a common i...,2011,Germany,https://doi.org/10.1186/2110-5820-1-43,"{""pmid"": 21970512, ""electronic_issn"": ""2110-58..."
3,84e99a2c-3267-4239-97b4-ab1d2b1a2d88,Frequency of upper gastrointestinal bleeding i...,"[{'author': 'Lacroix, J', 'author_abbreviated'...",Critical care medicine,20,1,35-42,OBJECTIVE: To determine the frequency of upper...,1992,United States,https://doi.org/10.1097/00003246-199201000-00013,"{""pmid"": 1729041, ""print_issn"": ""0090-3493"", ""..."
4,ff429945-9122-41d9-9adc-2b39a77cbc0f,Prehospital fluid management of abdominal orga...,"[{'author': 'Heuer, Matthias', 'author_abbrevi...",Langenbeck's archives of surgery,400,3,371-9,PURPOSE: Severe bleeding after trauma frequent...,2015,Germany,https://doi.org/10.1007/s00423-015-1274-2,"{""pmid"": 25681238, ""electronic_issn"": ""1435-24..."


In [24]:
# 必要なオブジェクトを準備
# df2, create_json_text, loaded_vectorizer, loaded_model, loaded_threshold を用意済みとする
updated_df, class_distribution, probability_stats = predict_and_summarize(
    df3, create_json_text, loaded_vectorizer, loaded_model, loaded_threshold
)


新しいデータの形状: (10404, 10000)
予測を実行し、結果をデータフレームに追加しました。

=== 予測結果 ===
                                                    tiab  prediction  \
0      {"title": "Association of transfusion red bloo...           1   
1      {"title": "Transfusion of packed red blood cel...           1   
2      {"title": "Red blood cell transfusion in the c...           1   
3      {"title": "Frequency of upper gastrointestinal...           0   
4      {"title": "Prehospital fluid management of abd...           0   
...                                                  ...         ...   
10399  {"title": "Guidelines on the management of ana...           1   
10400  {"title": "Blood transfusion and lung function...           1   
10401  {"title": "Syncope and epilepsy", "abstract": ...           0   
10402  {"title": "Anemia and blood transfusion in cri...           1   
10403  {"title": "Allogeneic umbilical cord blood red...           0   

       probability  
0         0.313773  
1         0.549822  
2     

In [25]:
print(df3.columns)

Index(['key', 'title', 'authors', 'journal', 'volume', 'issue', 'pages',
       'abstract', 'year', 'publisher', 'url', 'issn', 'tiab', 'prediction',
       'probability'],
      dtype='object')


In [26]:
df3["prediction"].value_counts()

prediction
0    8780
1    1624
Name: count, dtype: int64

In [27]:
df_export = df3[df3["prediction"] == 1]

In [28]:
df_export = df_export[['key', 'title', 'authors', 'journal', 'volume', 'issue', 'pages',
       'abstract', 'year', 'publisher', 'url', 'issn']]

In [29]:
df_export.shape

(1624, 12)

In [30]:
df_export.to_csv('2024-12-11exported_1624.csv', index=False)

In [43]:
# df_exportをランダムに500とそれ以外に分割して、それぞれをCSVファイルとしてエクスポート, seed設定
df_export.sample(frac=1, random_state=42).iloc[:500].to_csv('exported_data500.csv', index=False)
df_export.sample(frac=1, random_state=42).iloc[500:].to_csv('remaining_data10404.csv', index=False)

